In [185]:
import numpy as np
import pandas as pd
import math
import os
import xlrd
import matplotlib.pyplot as plt
from scipy.stats import linregress as lreg
pd.set_option('precision', 10)

In [70]:
propsdata=pd.read_csv('Historic_Secured_Property_Tax_Rolls.csv',low_memory=False)

In [5]:
coldescr = pd.ExcelFile('ASR-0001_DataDictionary_historic-secured-property-rolls.xlsx')

In [6]:
dfs = {sheet_name: coldescr.parse(sheet_name) for sheet_name in coldescr.sheet_names}

In [289]:
keys=dfs.keys()
propsdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1612110 entries, 206398 to 386172
Data columns (total 43 columns):
Closed Roll Fiscal Year                       1612109 non-null float64
Property Location                             1612110 non-null object
Neighborhood Code                             1611431 non-null object
Neighborhood Code Definition                  1564368 non-null object
Block and Lot Number                          1612110 non-null object
Volume Number                                 1612110 non-null int64
Property Class Code                           1611252 non-null object
Property Class Code Definition                1596776 non-null object
Year Property Built                           1483511 non-null float64
Number of Bathrooms                           1612110 non-null float64
Number of Bedrooms                            1612110 non-null int64
Number of Rooms                               1612110 non-null int64
Number of Stories                          

In [71]:
propsdata.columns.values

array(['Closed Roll Fiscal Year', 'Property Location', 'Neighborhood Code',
       'Neighborhood Code Definition', 'Block and Lot Number',
       'Volume Number', 'Property Class Code',
       'Property Class Code Definition', 'Year Property Built',
       'Number of Bathrooms', 'Number of Bedrooms', 'Number of Rooms',
       'Number of Stories', 'Number of Units',
       'Characteristics Change Date', 'Zoning Code', 'Construction Type',
       'Lot Depth', 'Lot Frontage', 'Property Area in Square Feet',
       'Basement Area', 'Lot Area', 'Lot Code', 'Prior Sales Date',
       'Recordation Date', 'Document Number', 'Document Number 2',
       'Tax Rate Area Code', 'Percent of Ownership',
       'Closed Roll Exemption Type Code',
       'Closed Roll Exemption Type Code Definition',
       'Closed Roll Status Code', 'Closed Roll Misc Exemption Value',
       'Closed Roll Homeowner Exemption Value', 'Current Sales Date',
       'Closed Roll Assessed Fixtures Value',
       'Closed Roll A

In [74]:
propsdata.sort_values(by=['Block and Lot Number','Closed Roll Fiscal Year'],inplace=True)

In [75]:
# Q2,p.1
classcode=propsdata["Property Class Code"]
classcode.dropna(inplace=True)
classcode.value_counts(normalize=True)[0]
# propsdata["Property Class Code"][propsdata["Property Class Code"].notnull()].value_counts(normalize=True)[0]

0.47097598637581212

In [79]:
# Q2,p.2
# propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')["Closed Roll Assessed Improvement Value"]\
# [propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')["Closed Roll Assessed Improvement Value"]!=0].median()
imprvlast=propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')["Closed Roll Assessed Improvement Value"]
imprvlast=imprvlast.dropna()
imprvlast=imprvlast[imprvlast!=0]
imprvlast.median()

209244.5

In [502]:
# Q2,p.3
# propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')[["Closed Roll Assessed Improvement Value","Neighborhood Code"]]\
# [propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')["Closed Roll Assessed Improvement Value"]!=0]
imprvlastnbr=propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')\
[["Closed Roll Assessed Improvement Value","Neighborhood Code"]]
imprvlastnbr=imprvlastnbr.dropna()
imprvlastnbr=imprvlastnbr[imprvlastnbr["Closed Roll Assessed Improvement Value"]!=0]
grnbr=imprvlastnbr.groupby(by='Neighborhood Code') #group by nbr
assimpvalgrpmean=grnbr.mean()['Closed Roll Assessed Improvement Value']
assimpvalgrpmean.max()-assimpvalgrpmean.min()

5085780.383029256

In [237]:
# Q2,p.4
ylv=propsdata[['Closed Roll Fiscal Year','Closed Roll Assessed Land Value']]
ylv=ylv.dropna()
ylv=ylv[ylv['Closed Roll Assessed Land Value']!=0]
y=ylv['Closed Roll Fiscal Year']
x=ylv['Closed Roll Assessed Land Value']
y=np.array(y)
x=np.array(x)
ylg=np.log(y)
xlg=np.log(x)
ylgc=ylg-ylg.mean()
xlgc=xlg-xlg.mean()
r=lreg(xlgc.T,ylgc)
# r[0]
# print('{:10.10f}'.format(r[0]))
# double check
r2=np.dot(xlgc,ylgc)/np.dot(xlgc,xlgc)
print('pckg: {:10.10f} mine: {:10.10f} '.format(r[0],r2))

pckg: 0.0000449280 mine: 0.0000449280 


In [585]:
# ln,lt to km
def measure(lat1, lon1, lat2, lon2):
    R = 6378.137; # Radius of earth in KM
    dLat = (lat2 - lat1) * np.pi / 180.0
    dLon = (lon2 - lon1) * np.pi / 180.0
    a = np.sin(dLat/2.0) * np.sin(dLat/2.0) +\
    np.cos(lat1 * np.pi / 180.0) * np.cos(lat2 * np.pi / 180.0) *\
    np.sin(dLon/2) * np.sin(dLon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c
    return d  #km

In [604]:
# Q2,p.5
# std measured in each nbr seperately
# consider only once each lot, take i.e. the last assessment
locsnbr=propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')
locsnbr=locsnbr[["Neighborhood Code","Location"]]
locsnbr=locsnbr.dropna()
locsnbr[["ln","lt"]]=locsnbr['Location'].str.replace('(','').str.replace(')','').str.split(',',expand=True)
locsnbr.drop('Location', axis=1, inplace=True)
locsnbr[["ln","lt"]]=locsnbr[["ln","lt"]].apply(lambda x: np.float128(x))

# group by nbr
grnbr=locsnbr.groupby(by='Neighborhood Code')

# stds=grnbr.transform(lambda x: x.std())
mn1=grnbr.mean()["lt"]
mn2=grnbr.mean()["ln"]

ax1=measure(mn1, mn2, mn1+grnbr.std()["lt"], mn2)
ax2=measure(mn1, mn2, mn1, mn2+grnbr.std()["ln"])

# areas=(np.pi*ax1*ax2)*np.float128(grnbr.count()["ln"])*6378.137**2
areas=np.pi*ax1*ax2
areas.max()

2.0589197506122123

In [768]:
# Q2,p.6
untyb=propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='first')[["Year Property Built","Number of Units"]]
untyb=untyb.dropna()
untyb=untyb[untyb["Number of Units"]!=0]
# removing outliers
y=untyb["Year Property Built"]
st=y.describe()
out1=y.median()-1.5*(st[6]-st[4])
out2=2016 #y.median()+1.5*(st[6]-st[4])
untyb=untyb[(y>out1)&(y<out2)]
# 
f = lambda x: 1900 if x<1950 else 1950
untyb["Year Property Built"]=untyb["Year Property Built"].apply(f)
# 
grnbr=untyb.groupby(by='Year Property Built')
# 
grnbr.mean()["Number of Units"][1950]-grnbr.mean()["Number of Units"][1900]
# bef=untyb[untyb["Year Property Built"]<1950]
# af=untyb[untyb["Year Property Built"]>=1950]

0.41975159872122658

In [780]:
#Q2,p.7
# Assumption: 1) I consider only the last report 2) I drop the outliers 3) I ignore 0 bedrroms and 0 units 4) remove NaN
# 1
beduntzip=propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')\
[['Zipcode of Parcel','Number of Bedrooms','Number of Units','Year Property Built']]
# 4
beduntzip=beduntzip.dropna()
# 2
# Removing outliers
y=beduntzip["Year Property Built"]
st=y.describe()
out1=y.median()-1.5*(st[6]-st[4])
out2=2016 #y.median()+1.5*(st[6]-st[4])
beduntzip=beduntzip[(y>out1)&(y<out2)]
beduntzip.drop('Year Property Built', axis=1, inplace=True)
# 3
beduntzip=beduntzip[beduntzip[['Number of Bedrooms','Number of Units']].sum(1)!=0]
# grouping
grnbr=beduntzip.groupby(by='Zipcode of Parcel')
(grnbr.mean()["Number of Bedrooms"]/grnbr.mean()["Number of Units"]).max()

2.4479166666666665

In [795]:
# Q2,p.8
# Assumption: 1) I consider only the last report 2) I drop the outliers 4) remove NaN
# 1
zipproparealotarea=propsdata.drop_duplicates(subset=["Block and Lot Number"],keep='last')\
[['Zipcode of Parcel','Property Area in Square Feet','Lot Area','Year Property Built']]
# 4
zipproparealotarea=zipproparealotarea.dropna()
# 2
# Removing outliers
y=zipproparealotarea["Year Property Built"]
st=y.describe()
out1=y.median()-1.5*(st[6]-st[4])
out2=2016 #y.median()+1.5*(st[6]-st[4])
zipproparealotarea=zipproparealotarea[(y>out1)&(y<out2)]
zipproparealotarea.drop('Year Property Built', axis=1, inplace=True)
# 
# grouping
grnbr=zipproparealotarea.groupby(by='Zipcode of Parcel')
(grnbr.sum()['Property Area in Square Feet']/grnbr.sum()['Lot Area']).max()

12.846970209624423